In [2]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import json
import pandas as pd
from itertools import combinations
import time
import threading

In [3]:
# authorization + connect to app

quick = False
max_files_for_quick_processing = 10

# Set up Spotify API credentials
client_id = ''
client_secret = ''

# Set up authentication with required scope
scope = 'playlist-modify-public, user-library-read, user-follow-read, user-top-read'  # Include necessary scope
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, scope=scope, redirect_uri="http://localhost:3000/callback"))

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd


# Function to retrieve user's top 3 songs of the month
def get_top_songs():
    top_tracks = sp.current_user_top_tracks(limit=3, time_range='short_term')
    return [track['uri'] for track in top_tracks['items']]

# Function to recommend songs based on top songs' URIs
def recommend_songs(top_song_uris):
    recommended_tracks = sp.recommendations(seed_tracks=top_song_uris, limit=7)
    return [(track['name'], track['artists'][0]['name']) for track in recommended_tracks['tracks']]

# Get user's top 3 songs of the month and extract their URIs
top_song_uris = get_top_songs()

# Recommend songs based on top song URIs
# recommended_songs = recommend_songs(top_song_uris)


In [5]:
import os
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import random

def process_pair(filename, path, target, pair_occurrences):
    fullpath = os.path.join(path, filename)
    with open(fullpath) as f:
        mpd_slice = json.load(f)

    for playlist in mpd_slice["playlists"]:
        tracks = pd.DataFrame(playlist['tracks'])

        if target in tracks[['track_name', 'artist_name']].values:
            for track in tracks[['track_name', 'artist_name']].values:
                if track[1] != target[1]:
                    key = tuple(track)
                    pair_occurrences[key] = pair_occurrences.get(key, 0) + 1

def pair_occurrences(target, path, pool_size):
    pair_occurrences = dict()
    filenames = [filename for filename in os.listdir(path) if filename.startswith("mpd.slice.") and filename.endswith(".json")]
    random.shuffle(filenames)
    pool = filenames[:pool_size]

    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = []
        for filename in pool:
            future = executor.submit(process_pair, filename, path, target, pair_occurrences)
            futures.append(future)

        for future in futures:
            future.result()

    return pd.DataFrame({'track': pair_occurrences.keys(), 'pair_occurrences': pair_occurrences.values()}), pool


In [6]:
import os
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from collections import Counter

def process_freq(filename, path):
    fullpath = os.path.join(path, filename)
    with open(fullpath) as f:
        mpd_slice = json.load(f)

    playlist_occurrences = Counter()
    for playlist in mpd_slice["playlists"]:
        playlist_tracks = [(track['track_name'], track['artist_name']) for track in playlist['tracks']]
        playlist_occurrences.update(playlist_tracks)

    return playlist_occurrences

def song_frequency(path, pool):
    frequencies = Counter()

    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(process_freq, filename, path) for filename in pool]

        for future in futures:
            result = future.result()
            frequencies.update(result)

    return pd.DataFrame({'track': frequencies.keys(), 'occurrences': frequencies.values()})

# Example usage:
# result = song_frequency(path, pool)
# print(result)


In [7]:
track_database = pd.DataFrame(columns=["track name", "artist name", "track popularity", "artist popularity", "monthly listeners", "track"])
artist_database = pd.DataFrame()

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Function to retrieve track and artist information
def get_track_artist_info(track):
    track_name, artist_name = track
    results = sp.search(q='track:{} artist:{}'.format(track_name, artist_name), type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_popularity = track['popularity']
        artist = track['artists'][0]
        artist_id = artist['id']
        artist_popularity, monthly_listeners = get_artist_info(artist_name, artist_id)
        return track_name, artist_name, track_popularity, artist_popularity, monthly_listeners
    else:
        return track_name, artist_name, None, None, None

# Function to retrieve artist information
def get_artist_info(artist_name, artist_id):
    if artist_name in artist_database['artist name']:
        return artist_database.loc[artist_database['artist name'] == artist_name]['artist popularity'], artist_database.loc[artist_database['artist name'] == artist_name]['monthly listeners']
    elif artist_name in new_artists:
        return new_artists[artist_name][0], new_artists[artist_name][1]
    else:
        artist_info = sp.artist(artist_id)
        artist_popularity = artist_info['popularity']
        monthly_listeners = artist_info['followers']['total']
        new_artists[artist_name] = (artist_popularity, monthly_listeners)
        return artist_popularity, monthly_listeners
    
new_artists = {}

artist_database = track_database[['artist name', 'artist popularity', 'monthly listeners']].drop_duplicates()

# replacement_set = set(test['track'].values)

# database_set = set(track_database['track'].values)

# track = list(replacement_set - database_set)

# print(len(track))

# # Retrieve track and artist information
# with ThreadPoolExecutor(max_workers=4) as executor:
#     results = list(executor.map(get_track_artist_info, track))

# # Create dataframe to store results
# df = pd.DataFrame(results, columns=["track name", "artist name", "track popularity", "artist popularity", "monthly listeners"])
# df['track'] = list(zip(df['track name'], df['artist name']))
# track_database = pd.concat([track_database, df])
# display(track_database)

In [18]:
path = "" #mpd location

# Get user's top 3 songs of the month and extract their URIs
top_song_uris = get_top_songs()
REPLACEMENT_PLAYLIST = []

print(top_song_uris)

# Recommend songs based on top song URIs
# recommended_songs = recommend_songs(top_song_uris)
recommended_songs = [('Reptilia', 'The Strokes'), 
 ('For the First Time', 'Mac DeMarco'), 
 ('All You Need Is Love - Remastered 2009', 'The Beatles'), 
 ("Work This Time", "King Gizzard & The Lizard Wizard"), 
 ("Cigarette Daydreams", "Cage The Elephant")]

replacements = dict.fromkeys(recommended_songs, None)

for song in recommended_songs:
    print(song)
    # find pair occurences
    target_pair, pool = pair_occurrences(song, path, pool_size=11)
    print('done pair occurrences')
    print(pool)

    if not target_pair.empty:
        # find frequency of all related songs
        target_freq = song_frequency(path, pool)

        # combine datasets
        target_stats = pd.merge(target_pair, target_freq, on='track', how='left')

        print(target_stats)

        if len(target_stats.loc[target_stats['pair_occurrences'] > 2]) > 100:
            print('good selection')
            # filter range
            target_stats = target_stats.loc[(target_stats['pair_occurrences'] > 2) & (target_stats['occurrences'] < 50)]

        # normalize
        target_stats['pair norm'] = (target_stats['pair_occurrences'] - target_stats['pair_occurrences'].min()) / (target_stats['pair_occurrences'].max() - target_stats['pair_occurrences'].min()) * 99 + 1
        target_stats['occ norm'] = (target_stats['occurrences'] - target_stats['occurrences'].min()) / (target_stats['occurrences'].max() - target_stats['occurrences'].min()) * 99 + 1

        # filter further
        target_stats['score'] = target_stats['pair norm']/ (target_stats['occ norm'])
        test = target_stats.sort_values(by='score', ascending = False).head(100)

        new_artists = {}

        artist_database = track_database[['artist name', 'artist popularity', 'monthly listeners']].drop_duplicates()

        replacement_set = set(test['track'].values)

        database_set = set(track_database['track'].values)

        track = list(replacement_set - database_set)

        print(len(track))

        # Retrieve track and artist information
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = list(executor.map(get_track_artist_info, track))

        # Create dataframe to store results
        df = pd.DataFrame(results, columns=["track name", "artist name", "track popularity", "artist popularity", "monthly listeners"])
        df['track'] = list(zip(df['track name'], df['artist name']))
        track_database = pd.concat([track_database, df])
        
        info = pd.merge(track_database, test, on='track', how='inner')
        info['monthly listeners normalized'] = (info['monthly listeners'] - info['monthly listeners'].min()) / (info['monthly listeners'].max() - info['monthly listeners'].min()) * 99 + 1

        # zero_rows = info[(info == 0).any(axis=1)]
        # print('bruh')
        # print(zero_rows)

        info['track popularity'] = info['track popularity'].replace(0.0, 1.0)
        info['artist popularity'] = info['artist popularity'].replace(0.0, 1.0)

        info['score'] = info['pair norm']/ (info['occ norm'])**2 / (info['track popularity']) / (info['artist popularity']) / info['monthly listeners normalized']

        sorted = info.loc[(info['track popularity'] > 20) & (info['artist popularity'] > 20)].sort_values(by='score', ascending = False)

        REPLACEMENT_PLAYLIST.append(sorted['track'].iloc[0])
        print(sorted['track'].iloc[0])

        # replacements[song] = sorted['track'].iloc[0]
        # print(sorted['track'].iloc[0])

# print(replacements)

['spotify:track:2zf1izCOz2F22PF27uhxRF', 'spotify:track:2s6Jh6GmDhAOsSMII07fVs', 'spotify:track:6nUPcVDYjCrnNMaXAJ9eO3']
('Reptilia', 'The Strokes')
done pair occurrences
['mpd.slice.18000-18999.json', 'mpd.slice.14000-14999.json', 'mpd.slice.15000-15999.json', 'mpd.slice.0-999.json', 'mpd.slice.8000-8999.json', 'mpd.slice.492000-492999.json', 'mpd.slice.493000-493999.json', 'mpd.slice.6000-6999.json', 'mpd.slice.40000-40999.json', 'mpd.slice.27000-27999.json', 'mpd.slice.47000-47999.json']
                                                   track  pair_occurrences  \
0                    (Little Talks, Of Monsters and Men)                16   
1                             (The Cave, Mumford & Sons)                 7   
2      (Awake My Soul - Live At Shepherd's Bush Empir...                 1   
3                                (Time to Pretend, MGMT)                19   
4                                       (Uprising, Muse)                 8   
...                                 

C:\Users\mlpui\AppData\Local\Temp\ipykernel_19800\1405412345.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  info['track popularity'] = info['track popularity'].replace(0.0, 1.0)
C:\Users\mlpui\AppData\Local\Temp\ipykernel_19800\1405412345.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  info['artist popularity'] = info['artist popularity'].replace(0.0, 1.0)


('Thank You Neil', 'Dumbo Gets Mad')
('Cigarette Daydreams', 'Cage The Elephant')
done pair occurrences
['mpd.slice.24000-24999.json', 'mpd.slice.20000-20999.json', 'mpd.slice.499000-499999.json', 'mpd.slice.491000-491999.json', 'mpd.slice.49000-49999.json', 'mpd.slice.496000-496999.json', 'mpd.slice.3000-3999.json', 'mpd.slice.502000-502999.json', 'mpd.slice.26000-26999.json', 'mpd.slice.1000-1999.json', 'mpd.slice.21000-21999.json']
                                         track  pair_occurrences  occurrences
0            (The Original High, Adam Lambert)                 1            3
1          (Things I Didn't Say, Adam Lambert)                 1            3
2                      (Brand New, Ben Rector)                 5           37
3            (Fairly Local, Twenty One Pilots)                 7           52
4                 (Starving, Hailee Steinfeld)                 7          165
...                                        ...               ...          ...
18628          

In [20]:
# original playlist
original_playlist = [('Reptilia', 'The Strokes'), 
 ('For the First Time', 'Mac DeMarco'), 
 ('All You Need Is Love - Remastered 2009', 'The Beatles'), 
 ("Work This Time", "King Gizzard & The Lizard Wizard"), 
 ("Cigarette Daydreams", "Cage The Elephant")]

# new playlist
new_playlist_1 = [("Bang Bang You're Dead", 'Dirty Pretty Things'), 
 ('For the First Time', 'Kenny Chesney'), 
 ('Calling All Angels (with k.d. lang)', 'Jane Siberry'), 
 ("Better Day", "STS9"), 
 ("Regular World", "Spirit Animal")]

new_playlist_2 = [('New Fang', 'Them Crooked Vultures'),
 ('First Time', 'Lifehouse'),
 ('Everybody Just Wants to Dance', 'Kris Allen'),
 ('Blue Mountains', 'Diamond Rugs'),
 ('Doing What They Taught Us', 'Rambutan Jam Band')]

new_playlist_3 = [("We Can't Be Beat", 'The Walkmen'),
 ('This Is Our Moment', 'Kenny Chesney'),
 ('Here Comes My Baby', 'The Tremeloes'),
 ('Baby Boy', 'Pile'),
 ('Bittersweet Symphony', 'The Aranbee Pop Symphony Orchestra')]

new_playlist_4 = [('Post Acid', 'Wavves'),
 ('Dead Man Walking', 'The Script'),
 ('Run', 'Tiggs Da Author'),
 ("It's You", 'Robert Schwartzman'),
 ('Balance', 'Opus Orange')]

new_playlist_5 = [("Mama's Room", 'Under The Influence of Giants'),
 ('Hands', 'Jewel'),
 ('Calling All Angels (with k.d. lang)', 'Jane Siberry'),
 ('Thank You Neil', 'Dumbo Gets Mad'),
 ('Bury', 'Unions')]


In [ ]:
# original playlist
original_playlist = [('Reptilia', 'The Strokes'), 
 ('For the First Time', 'Mac DeMarco'), 
 ('All You Need Is Love - Remastered 2009', 'The Beatles'), 
 ("Work This Time", "King Gizzard & The Lizard Wizard"), 
 ("Cigarette Daydreams", "Cage The Elephant")]

# new playlist
new_playlist = [original_playlist, new_playlist_1, new_playlist_2, new_playlist_3, new_playlist_4, new_playlist_5]

new_song_data = pd.DataFrame(columns=["track name", "artist name", "track popularity", "artist popularity", "artist monthly listeners", "artist genres", "release date", "playlist num"])
counter = 0

for playlist in new_playlist:
    
    for song, artist in playlist:
        # Search for the track
        query = f'track:{song} artist:{artist}'
        result = sp.search(q=query, type='track', limit=1)
        if result['tracks']['items']:
            track = result['tracks']['items'][0]
            track_name = track['name']
            artist_name = track['artists'][0]['name']
            track_popularity = track['popularity']
            artist_id = track['artists'][0]['id']
            release_date = track['album']['release_date']
            
            # Get artist info
            artist_info = sp.artist(artist_id)
            artist_popularity = artist_info['popularity']
            artist_genres = artist_info['genres']
            
            # artist monthly listeners is NOT available via API
            artist_monthly_listeners = artist_info['followers']['total']

            # Append to DataFrame
            new_song_data = pd.concat([new_song_data, pd.DataFrame([{
                "track name": track_name,
                "artist name": artist_name,
                "track popularity": track_popularity,
                "artist popularity": artist_popularity,
                "artist monthly listeners": artist_monthly_listeners,
                "artist genres": artist_genres,
                "release date": release_date,
                "playlist num": counter
            }])], ignore_index=True)
        else:
            print(f"Track not found: {song} by {artist}")

    counter = counter+1  

display(new_song_data)
